In [15]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('AOAI_API_KEY')
openai.api_type = "azure"
openai.api_base = os.getenv('AOAI_ENDPOINT')
openai.api_version = "2023-05-15"

def get_completion_from_messages(messages,temperature=0):
    response = openai.ChatCompletion.create(
        engine=os.getenv('AOAI_MODEL'),
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )

    print(response.usage)

    return response.choices[0].message["content"]

In [10]:
# CSVファイルの読み込み
import pandas as pd
df = pd.read_csv('../private/testdata.csv', encoding='utf-8')

# 件名と本文と差出人 : (住所)のみに絞る
df = df[['件名', '本文', '差出人 : (住所)']]
#df.head()

# 2行目の"本文”を取得
text = df.iloc[1,1]

text


'こんにちは。洗濯機が壊れたことに関する相談、ありがとうございました。 \r\n\r\n洗濯機が壊れてしまったことは大変申し訳ありません。 保証期間内ですので、無償で修理させていただきます。 \r\n\r\n修理の手順は以下の通りです。\r\n\r\n1.\tお客様のお住まいの地域にある修理センターにお電話ください。電話番号は以下のリンクからご確認いただけます。\r\n2.\t修理センターの担当者が、洗濯機の型番と故障状況をお伺いします。また、修理日時と修理費用（無料）をお知らせします。\r\n3.\t修理日時に合わせて、修理センターから派遣された技術者がお客様のご自宅に伺います。技術者が洗濯機を点検し、修理します。\r\n4.\t修理が完了したら、技術者が動作確認を行います。また、修理内容と保証書をお渡しします。\r\n5.\t修理後も万が一不具合が発生した場合は、再度修理センターにご連絡ください。迅速に対応させていただきます。\r\n\r\n\r\n\r\n以上が修理の手順です。ご不明な点がありましたら、お気軽にお問い合わせください。 ご迷惑をおかけして申し訳ありませんが、何卒よろしくお願いします。\r\n\r\n2023年6月22日(木) 12:19 Shuji Yamaguchi (CSA DEV ADVOCATE) <Shuji.Yamaguchi@microsoft.com <mailto:Shuji.Yamaguchi@microsoft.com> >:\r\n\r\n\r\n\tこんにちは。先日購入した洗濯機が壊れてしまいました。\r\n\r\n\t洗濯中に異音がして止まってしまい、再起動もできませんでした。\r\n\r\n\t保証期間内ですが、修理にはどうすればいいでしょうか？ \r\n\r\n\t返信をお待ちしています。 よろしくお願いします。\r\n\r\n'

In [14]:
system_prompt = """あなたは日本語テキスト文章解析気です。
入力されるメールのやりとりに対して、文章を読取、以下のルールで段落分けをします。
ルール:
 - 2つ以上続く余計な改行や空白は削除します。
 - 差出人が変わる場所で、段落を分けます。
 - 同じ差出人の文章でも、内容が変わった場合は、段落を分けます。
"""

system_prompt = """あなたは日本語テキスト文章解析器です。
入力されるメールのやりとりに対して、文章を読取、以下のルールで文章を分割します。
なお、分割した結果は、以下の出力フォーマットに従ってJSONのみ回答してください。
*説明や補足などは不要です。JSONのみ回答してください。

ルール:
 - 2つ以上続く余計な改行や空白は削除します。
 - 差出人が変わる場所で、文章を分割します。
 - 最後のメールが途中で切れている場合は、そのメールの文章は余りとして出力します。

出力フォーマット:
{
    paragraphs: ["メール1","メール2"],
    surplus: "メール3途中まで"
}
"""

user_prompt = text

messages =  [
{'role':'system', 'content':system_prompt},
{'role':'user', 'content':user_prompt},
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

{
  "id": "chatcmpl-7U71J6ynZLomkKo8UsCnuzeaOze1I",
  "object": "chat.completion",
  "created": 1687411181,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "{\n \"paragraphs\": [\n  \"\u3053\u3093\u306b\u3061\u306f\u3002\u6d17\u6fef\u6a5f\u304c\u58ca\u308c\u305f\u3053\u3068\u306b\u95a2\u3059\u308b\u76f8\u8ac7\u3001\u3042\u308a\u304c\u3068\u3046\u3054\u3056\u3044\u307e\u3057\u305f\u3002\\n\\n\u6d17\u6fef\u6a5f\u304c\u58ca\u308c\u3066\u3057\u307e\u3063\u305f\u3053\u3068\u306f\u5927\u5909\u7533\u3057\u8a33\u3042\u308a\u307e\u305b\u3093\u3002\u4fdd\u8a3c\u671f\u9593\u5185\u3067\u3059\u306e\u3067\u3001\u7121\u511f\u3067\u4fee\u7406\u3055\u305b\u3066\u3044\u305f\u3060\u304d\u307e\u3059\u3002\\n\\n\u4fee\u7406\u306e\u624b\u9806\u306f\u4ee5\u4e0b\u306e\u901a\u308a\u3067\u3059\u3002\\n\\n1.\\t\u304a\u5ba2\u69d8\u306e\u304a\u4f4f\u307e\u3044\u306e\u5730\u57df\u306b\u3042\u308b\u4fe